In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
test = pd.read_csv("../input/emotions-dataset-for-nlp/test.txt", header=None, sep=";", names=["text","mood"], encoding="utf-8")
train= pd.read_csv("../input/emotions-dataset-for-nlp/train.txt", header=None, sep=";", names=["text","mood"], encoding="utf-8")
val= pd.read_csv("../input/emotions-dataset-for-nlp/val.txt", header=None, sep=";", names=["text","mood"], encoding="utf-8")


In [ ]:
print("Train : ", train.shape)
print("Test : ", test.shape)
print("Validation : ", val.shape)

In [ ]:
'''import seaborn as sns
sns.countplot((train['text']))
plt.show'''

In [ ]:
train["length"] = [len(i) for i in train["text"]]

In [ ]:
from sklearn.preprocessing import LabelEncoder
lb = LabelEncoder()
train["mood"] = lb.fit_transform(train["mood"])
test["mood"] = lb.fit_transform(test["mood"])
val["mood"] = lb.fit_transform(val["mood"])

In [ ]:
import re 
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

In [ ]:
vocab_size = 10000
train["length"].max()
train["length"].min()
len_sentence = 150

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.preprocessing.sequence import pad_sequences
import nltk
nltk.download('stopwords')
stop_words = set(nltk.corpus.stopwords.words('english'))
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import re

In [ ]:
def text_prepare(data, column):
    print(data.shape)
    stemmer = PorterStemmer()
    corpus = []
    
    for text in data[column]:
        text = re.sub("[^a-zA-Z]", " ", text)
        
        text = text.lower()
        text = text.split()
        
        text = [stemmer.stem(word) for word in text if word not in stop_words]
        text = " ".join(text)
        
        corpus.append(text)
    one_hot_word = [one_hot(input_text=word, n=vocab_size) for word in corpus]
    embeddec_doc = pad_sequences(sequences=one_hot_word,
                              maxlen=len_sentence,
                              padding="pre")
    print(data.shape)
    return embeddec_doc

In [ ]:
x_train=text_prepare(train, "text")
x_validate=text_prepare(val, "text")
x_test=text_prepare(test, "text")

In [ ]:
y_train=train["mood"]
y_validate=val["mood"]
y_test=test["mood"]

In [ ]:
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder()
y_train = np.array(y_train)
y_train = enc.fit_transform(y_train.reshape(-1,1)).toarray()

In [ ]:
y_test = np.array(y_test)
y_validate = np.array(y_validate)

y_test = enc.fit_transform(y_test.reshape(-1,1)).toarray()
y_validate = enc.fit_transform(y_validate.reshape(-1,1)).toarray()

In [ ]:
import tensorflow as tf
import tensorflow.keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.optimizers import Adam

In [ ]:
import tensorflow as tf
model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=150, input_length=len_sentence))
model.add(Dropout(0.2))
model.add(LSTM(128))
model.add(Dropout(0.2))
model.add(Dense(64, activation="sigmoid"))
model.add(Dropout(0.2))
model.add(Dense(6, activation="softmax"))

In [ ]:
model.compile(optimizer="Adam", loss = "categorical_crossentropy", metrics=["accuracy"])

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint


es = EarlyStopping(monitor = 'val_loss', mode = 'min', verbose = 1, patience = 5)
mc = ModelCheckpoint('./model.h5', monitor = 'val_accuracy', mode = 'max', verbose = 1, save_best_only = True)

In [ ]:
model.summary()

In [ ]:
hist = model.fit(x_train, y_train, epochs = 5, batch_size = 64, validation_data=(x_validate, y_validate),verbose = 1, callbacks= [es, mc])

In [ ]:
## mood
import textblob
from textblob import TextBlob
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
stop_words = set(nltk.corpus.stopwords.words('english'))


In [ ]:
def moods(score):
    if score > 0:
        return 'Positive'
    elif score < 0:
        return 'Negative'
    else:
        return 'Neutral'

In [ ]:
train['text_2'] = train['text'].apply(lambda x: TextBlob(x).words)
train['selected_text'] = train['text_2'].apply(lambda x: [w for w in x if w not in (stop_words)])
train['emotion'] = train['selected_text'].apply(lambda x: TextBlob(str(x)).sentiment.polarity)
train['emotion']=train['emotion'].apply(moods)
train.head()

In [ ]:
train['mood'].value_counts()

In [ ]:
'''sns.counterplot((train['emotion']))
plt.show'''

In [ ]:
test['text_2'] = test['text'].apply(lambda x: TextBlob(x).words)
test['selected_text'] = test['text_2'].apply(lambda x: [w for w in x if w not in (stop_words)])
test['moods'] = test['selected_text'].apply(lambda x: TextBlob(str(x)).sentiment.polarity)
test['moods']=test['moods'].apply(mood)
test.head()

In [ ]:
test['mood'].value_counts()

In [ ]:
'''sns.counterplot((test['emptionss']))
plt.show()'''

In [ ]:
val['text_3'] = val['text'].apply(lambda x: TextBlob(x).words)
val['selected_text'] = val['text_3'].apply(lambda x: [w for w in x if w not in (stop_words)])
val['mood'] = val['selected_text'].apply(lambda x: TextBlob(str(x)).sentiment.polarity)
val['mood']=val['mood'].apply(mood)
val.head()

In [ ]:
val['mood'].value_counts()

In [ ]:
'''sns.counterplot((val['mood']))
plt.show()'''

In [ ]:
train_csv = pd.DataFrame(train)
train_csv.to_csv('submission_train.csv')

In [ ]:
test_csv = pd.DataFrame(test)
test_csv.to_csv('submission_test.csv')

In [ ]:
val_csv = pd.DataFrame(val)
val_csv.to_csv('submission_val.csv')